![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/Certification_Trainings/Healthcare/30.1.Text_Classification_with_DocumentMLClassifier.ipynb)

# 30.1 Text Classification with DocumentMLClassifier

## Colab Setup

In [ ]:
import json
import os

from google.colab import files

if 'spark_jsl.json' not in os.listdir():
  license_keys = files.upload()
  os.rename(list(license_keys.keys())[0], 'spark_jsl.json')

with open('spark_jsl.json') as f:
    license_keys = json.load(f)

# Defining license key-value pairs as local variables
locals().update(license_keys)
os.environ.update(license_keys)

In [ ]:
# Installing pyspark and spark-nlp
% pip install --upgrade -q pyspark==3.4.1 spark-nlp==$PUBLIC_VERSION

# Installing Spark NLP Healthcare
% pip install --upgrade -q spark-nlp-jsl==$JSL_VERSION  --extra-index-url https://pypi.johnsnowlabs.com/$SECRET

In [ ]:
import pandas as pd
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score

from pyspark.ml import Pipeline,PipelineModel
from pyspark.sql import SparkSession

from sparknlp.annotator import *
from sparknlp_jsl.annotator import *
from sparknlp_jsl.base import *
from sparknlp.base import *
import sparknlp_jsl
import sparknlp

import warnings
warnings.filterwarnings('ignore')

params = {"spark.driver.memory":"12G",
          "spark.kryoserializer.buffer.max":"2000M",
          "spark.driver.maxResultSize":"2000M"}

print ("Spark NLP Version :", sparknlp.version())
print ("Spark NLP_JSL Version :", sparknlp_jsl.version())

spark = sparknlp_jsl.start(license_keys['SECRET'],params=params)

spark

Spark NLP Version : 5.3.1
Spark NLP_JSL Version : 5.3.1


## 🔎 Models :


|index|model|
|-----:|:-----|
|1|[classifierml_ade](https://nlp.johnsnowlabs.com/2023/05/16/classifierml_ade_en.html)


## Load ADE Classification Dataset

In [ ]:
#downloading sample datasets
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/ADE-NEG.txt
!wget -q https://raw.githubusercontent.com/JohnSnowLabs/spark-nlp-workshop/master/tutorials/Certification_Trainings/Healthcare/data/ADE_Corpus_V2/DRUG-AE.rel

**ADE-Negative Dataset**

In [ ]:
df_neg= pd.read_csv("ADE-NEG.txt", header=None, delimiter="\t", names=["col1"])
df_neg['text'] =  df_neg.col1.str.split('NEG').str[1]
df_neg["category"] = "neg"
df_neg= df_neg[["text", "category"]]
df_neg.head()

,text,category
0,Clioquinol intoxication occurring in the trea...,neg
1,"""Retinoic acid syndrome"" was prevented with s...",neg
2,BACKGROUND: External beam radiation therapy o...,neg
3,"Although the enuresis ceased, she developed t...",neg
4,A 42-year-old woman had uneventful bilateral ...,neg


**ADE-Positive Dataset**

In [ ]:
df_pos= pd.read_csv("DRUG-AE.rel", header=None, delimiter="|")
df_pos["category"]= "pos"
df_pos.rename(columns={1: "text"}, inplace=True)
df_pos= df_pos[["text", "category"]]
df_pos.head()

,text,category
0,Intravenous azithromycin-induced ototoxicity.,pos
1,"Immobilization, while Paget's bone disease was...",pos
2,Unaccountable severe hypercalcemia in a patien...,pos
3,METHODS: We report two cases of pseudoporphyri...,pos
4,METHODS: We report two cases of pseudoporphyri...,pos


**Merging positive and negative datasets**

In [ ]:
ade_df= pd.concat([df_neg, df_pos]).sample(frac=1) #merge and shuffle the data
ade_df.head()

,text,category
1913,Carbamazepine toxicity following Oxybutynin an...,pos
7851,A proliferation test and an evaluation of TNF...,neg
7397,The trunk was twisted just after the appearan...,neg
3577,These reactions recurred immediately after th...,neg
9417,Complete relief of pain was noted in 73% of p...,neg


In [ ]:
ade_df["category"].value_counts()

neg    16695
pos     6821
Name: category, dtype: int64

In [ ]:
ade_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 23516 entries, 1913 to 8212
Data columns (total 2 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   text      23516 non-null  object
 1   category  23516 non-null  object
dtypes: object(2)
memory usage: 551.2+ KB


In [ ]:
spark_df = spark.createDataFrame(ade_df)
spark_df.head(3)

[Row(text='Carbamazepine toxicity following Oxybutynin and Dantrolene administration: a case report.', category='pos'),
 Row(text=' A proliferation test and an evaluation of TNF-alpha and interleukin (IL)-6 production from PBMC in presence of insulin were studied.', category='neg'),
 Row(text=' The trunk was twisted just after the appearance of the abdominal myoclonus associated with myoclonic jerks spreading from the rostral to caudal paraspinal muscles.', category='neg')]

In [ ]:
spark_df.groupBy("category").count().show()
spark_df.printSchema()

+--------+-----+
|category|count|
+--------+-----+
|     pos| 6821|
|     neg|16695|
+--------+-----+

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)



**Get train & test set**

In [ ]:
train_data, test_data = spark_df.randomSplit([0.8, 0.2], seed = 100)

print("Train Dataset Count: " + str(train_data.count()))
print("Test Dataset Count: " + str(test_data.count()))

Train Dataset Count: 18793
Test Dataset Count: 4723


## DocumentMLClassifier with Linear SVM

The below pipeline uses the SVM classifier of `DocumentMLClassifier` with setting `setClassificationModelClass("svm")`.

In [ ]:
document = DocumentAssembler().setInputCol("text").setOutputCol("document")

token = Tokenizer().setInputCols("document").setOutputCol("token")

stemmer = Stemmer() \
    .setInputCols(["token"]) \
    .setOutputCol("stem")

classifier_svm = DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")

pipeline = Pipeline(stages=[document, token, stemmer, classifier_svm])

In [ ]:
svm_model = pipeline.fit(train_data)
result_svm = svm_model.transform(test_data).cache()
result_svm.show()

+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
|                text|category|            document|               token|                stem|          prediction|
+--------------------+--------+--------------------+--------------------+--------------------+--------------------+
| A 12-year-old, s...|     neg|[{document, 0, 14...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 14...|
| A 40-year-old Ca...|     neg|[{document, 0, 16...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 16...|
| A 59-year-old wh...|     neg|[{document, 0, 18...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 18...|
| A 67-year-old ma...|     neg|[{document, 0, 71...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 71...|
| A biopsy of the ...|     neg|[{document, 0, 11...|[{token, 1, 1, A,...|[{token, 1, 1, a,...|[{category, 1, 11...|
| A case of isolat...|     neg|[{document, 0, 86...|[{token, 1, 1, A,...

In [ ]:
result_svm.printSchema()

root
 |-- text: string (nullable = true)
 |-- category: string (nullable = true)
 |-- document: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- va

In [ ]:
result_svm.select("text","prediction").show(4, truncate=100)

+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
|                                                                                                text|                                                                      prediction|
+----------------------------------------------------------------------------------------------------+--------------------------------------------------------------------------------+
| A 12-year-old, spayed female keeshond presenting with lethargy, anorexia, weight loss, and vomit...|[{category, 1, 141, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9822}, []}]|
| A 40-year-old Caucasian female was first seen 20 years ago for a routine ocular screening in rel...|[{category, 1, 167, neg, {sentence -> 0, chunk -> 0, confidence -> 0.9497}, []}]|
| A 59-year-old white man developed keratoacanthoma of the nose immediately foll

In [ ]:
result_svm_df = result_svm.select('category','prediction.result').toPandas()
result_svm_df['result'] = result_svm_df.result.apply(lambda x : x[0])

print (classification_report(result_svm_df['category'], result_svm_df['result']))

              precision    recall  f1-score   support

         neg       0.89      0.90      0.90      3381
         pos       0.75      0.73      0.74      1342

    accuracy                           0.85      4723
   macro avg       0.82      0.82      0.82      4723
weighted avg       0.85      0.85      0.85      4723



## DocumentMLClassifier with Logistic Regression

In [ ]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\

pipeline = Pipeline(stages=[document, token, classifier_logreg])

logreg_model = pipeline.fit(train_data)
result_logreg = logreg_model.transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      3381
         pos       0.78      0.73      0.75      1342

    accuracy                           0.86      4723
   macro avg       0.84      0.83      0.83      4723
weighted avg       0.86      0.86      0.86      4723



## Playing with Some Parameters

In [ ]:
classifier_logreg.extractParamMap()

{Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='lazyAnnotator', doc='Whether this AnnotatorModel acts as lazy in RecursivePipelines'): False,
 Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='labelCol', doc='column with the value result we are trying to predict.'): 'category',
 Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='maxIter', doc='maximum number of iterations.'): 10,
 Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='tol', doc='convergence tolerance after each iteration.'): 1e-06,
 Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='fitIntercept', doc='whether to fit an intercept term, default is true.'): True,
 Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='vectorizationModelPath', doc='specify the vectorization model if it has been already trained.'): '',
 Param(parent='DocumentMLClassifierApproach_e0842413ec1b', name='classificationModelPath', doc='specify the classification model if 

**Change `maxIter` parameter with `setMaxIter()`**

In [ ]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxIter(5)

pipeline = Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.88      0.95      0.92      3381
         pos       0.85      0.68      0.76      1342

    accuracy                           0.88      4723
   macro avg       0.87      0.82      0.84      4723
weighted avg       0.87      0.88      0.87      4723



**Change `tol` (tolarance) parameter with `setTol()`**

In [ ]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setTol(1e-3)

pipeline = Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_df = result_logreg.select('category','prediction.result').toPandas()
result_logreg_df['result'] = result_logreg_df.result.apply(lambda x : x[0])

print (classification_report(result_logreg_df['category'], result_logreg_df['result']))

              precision    recall  f1-score   support

         neg       0.90      0.92      0.91      3381
         pos       0.78      0.73      0.75      1342

    accuracy                           0.86      4723
   macro avg       0.84      0.83      0.83      4723
weighted avg       0.86      0.86      0.86      4723



**Change `maxTokenNgram` parameter with `setMaxTokenNgram()`**

In [ ]:
classifier_logreg = DocumentMLClassifierApproach() \
    .setInputCols("token") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("logreg")\
    .setMaxTokenNgram(2)

pipeline = Pipeline(stages=[document, token, classifier_logreg])

result_logreg = pipeline.fit(train_data).transform(test_data).cache()

result_logreg_pf = result_logreg.select('category','prediction.result').toPandas()
result_logreg_pf['result'] = result_logreg_pf.result.apply(lambda x : x[0])

print (classification_report(result_logreg_pf['category'], result_logreg_pf['result']))

              precision    recall  f1-score   support

         neg       0.91      0.92      0.91      3381
         pos       0.80      0.76      0.78      1342

    accuracy                           0.88      4723
   macro avg       0.85      0.84      0.85      4723
weighted avg       0.88      0.88      0.88      4723



## Using More Annotators

In [ ]:
document = DocumentAssembler() \
    .setInputCol("text") \
    .setOutputCol("document")

token = Tokenizer() \
  .setInputCols(["document"]) \
  .setOutputCol("token")

normalizer = Normalizer() \
    .setInputCols(["token"]) \
    .setOutputCol("normalized")

stopwords_cleaner = StopWordsCleaner()\
      .setInputCols("normalized")\
      .setOutputCol("cleanTokens")\
      .setCaseSensitive(False)

stemmer = Stemmer() \
    .setInputCols(["cleanTokens"]) \
    .setOutputCol("stem")

classifier_svm = DocumentMLClassifierApproach() \
    .setInputCols("stem") \
    .setLabelCol("category") \
    .setOutputCol("prediction") \
    .setClassificationModelClass("svm")\
    .setMaxIter(5)

finisher = Finisher() \
    .setInputCols(["prediction"]) \
    .setOutputCols(["predictions"]) \
    .setCleanAnnotations(True)\



pipeline = Pipeline(stages=[document, token, normalizer, stopwords_cleaner, stemmer, classifier_svm, finisher])

model= pipeline.fit(train_data)

In [ ]:
sample_df = spark.createDataFrame([["We report two cases of pseudoporphyria caused by naproxen and oxaprozin."],
                                   ["Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities."]]).toDF("text")

result = model.transform(sample_df)
result.show(truncate=False)

+--------------------------------------------------------------------------------------------------------------------+-----------+
|text                                                                                                                |predictions|
+--------------------------------------------------------------------------------------------------------------------+-----------+
|We report two cases of pseudoporphyria caused by naproxen and oxaprozin.                                            |[pos]      |
|Special attention should be paid when attempting to sample the endometrium in patients with mullerian abnormalities.|[neg]      |
+--------------------------------------------------------------------------------------------------------------------+-----------+

